In [1]:


from os import listdir
import json
import pandas as pd
import networkx as nx
import os
import csv

## Data Preparation

In [2]:
from sklearn.neighbors.tests.test_dist_metrics import n2, n1
input_dir = "annotazioni_revisionate/vecchie/"
raters={}


for file in listdir(input_dir):
    print(file)
    name=os.path.splitext(os.path.basename(file))[0]
    print("File: ", name)
    # raters.append(name)

    # use only columns ["prerequisite", "advanced", "weight", "agreem", "revised"]
    curr_df = pd.read_csv(input_dir + file, sep=",", encoding="utf-8", usecols=[1, 2, 4, 5, 9])
    raters[name]=curr_df

    # keep only relation with agreemen > 1 or those with agreement 1 but still present after revision
    curr_df = curr_df[(curr_df["agreem"] > 1) | (curr_df["Revised"].isin(["0.5", "0,5", "1", 1, 0.5]) ) ]
    print("\tNum of relations:", curr_df.shape[0], "\n")
    #for i, rel in curr_df.iterrows():
        # if rel['prerequisite'] in concepts and rel['advanced'] in concepts:

Andrea_Unique.csv
File:  Andrea_Unique
	Num of relations: 233 

Samu.csv
File:  Samu
	Num of relations: 368 

Chiara.csv
File:  Chiara
	Num of relations: 275 

Frosi.csv
File:  Frosi
	Num of relations: 392 

Marco.csv
File:  Marco
	Num of relations: 253 

Ilenia.csv
File:  Ilenia
	Num of relations: 87 



## Graph

In [3]:
def create_graphs(dataset):
    """
    Returns a networkx graph from a pandas dataframe with columns ["prerequisite", "advanced"]
    """

    G_nx = nx.DiGraph()

    for i, r in dataset.iterrows():
        G_nx.add_node(r["prerequisite"],label=r["prerequisite"])
        G_nx.add_node(r["advanced"],label=r["advanced"])
        G_nx.add_edge(r["prerequisite"], r["advanced"])

    return G_nx

## Calculating Vertext Edge Overlap

In [4]:
def veo_metric(raters):
    sim_veo = {}
    for rater1 in raters:
        for rater2 in raters:
            num_vertex_1=[]
            num_vertex_2=[]
            edge_intersection = 0

            if rater1 != rater2:
                rater1_graph = create_graphs(raters[rater1])
                rater2_graph = create_graphs(raters[rater2])
                vertexes_1 = set(set(raters[rater1]['prerequisite'] + raters[rater1]['advanced']))
                vertexes_2 = set(set(raters[rater2]['prerequisite'] + raters[rater2]['advanced']))
                vertex_intersection = vertexes_1.intersection(vertexes_2)
                print('#vertex: ', rater1, len(vertexes_1), rater2, len(vertexes_2))
                print('intersection #vertex: ', rater1, rater2, len(vertex_intersection))
                num_edges_1 = rater1_graph.number_of_edges()
                num_edges_2 = rater2_graph.number_of_edges()
                for e_1 in rater1_graph.edges:
                    for e_2 in rater2_graph.edges:
                        # print('PR_1: ',e_1)
                        # print('PR_2: ',e_2)
                        if e_1==e_2:
                            edge_intersection+=1
                sim_veo[rater1+rater2]=2*(len(vertex_intersection)+edge_intersection)/(len(vertexes_1)+len(vertexes_2)+num_edges_1+num_edges_2)

            #print('edge_intersection',edge_intersection)
            print(sim_veo)
            a_file = open("sim_veo.json", "w")
            json.dump(sim_veo, a_file)

In [5]:
veo_metric(raters)


{}
#vertex:  Andrea_Unique 279 Samu 435
intersection #vertex:  Andrea_Unique Samu 98
{'Andrea_UniqueSamu': 0.27450980392156865}
#vertex:  Andrea_Unique 279 Chiara 274
intersection #vertex:  Andrea_Unique Chiara 88
{'Andrea_UniqueSamu': 0.27450980392156865, 'Andrea_UniqueChiara': 0.3182640144665461}
#vertex:  Andrea_Unique 279 Frosi 410
intersection #vertex:  Andrea_Unique Frosi 83
{'Andrea_UniqueSamu': 0.27450980392156865, 'Andrea_UniqueChiara': 0.3182640144665461, 'Andrea_UniqueFrosi': 0.2409288824383164}
#vertex:  Andrea_Unique 279 Marco 275
intersection #vertex:  Andrea_Unique Marco 72
{'Andrea_UniqueSamu': 0.27450980392156865, 'Andrea_UniqueChiara': 0.3182640144665461, 'Andrea_UniqueFrosi': 0.2409288824383164, 'Andrea_UniqueMarco': 0.259927797833935}
#vertex:  Andrea_Unique 279 Ilenia 91
intersection #vertex:  Andrea_Unique Ilenia 32
{'Andrea_UniqueSamu': 0.27450980392156865, 'Andrea_UniqueChiara': 0.3182640144665461, 'Andrea_UniqueFrosi': 0.2409288824383164, 'Andrea_UniqueMarco': 

## Graph editi distance

In [ ]:
rater1_graph = create_graphs(raters['Chiara'])
print(type(rater1_graph))
rater2_graph = create_graphs(raters['Marco'])
print(type(rater2_graph))
optimized_edit_distance=0
edit_distance=0
# degree=nx.degree_histogram(rater1_graph)
# sp = dict(nx.all_pairs_shortest_path(rater1_graph))
# pagerank=nx.pagerank(rater1_graph)

# Ci mette troppo e non riesco ad arrivarci ad un risultato.
# edit_distance=nx.graph_edit_distance(rater1_graph, rater2_graph, node_match=lambda node1, node2:node1['label']==node2['label'], upper_bound=396)
for v in nx.optimize_graph_edit_distance(rater1_graph, rater2_graph, node_match=lambda node1, node2:node1['label']==node2['label']):
    print(v)
    optimized_edit_distance=v
print("optimized_edit_distance",optimized_edit_distance)
print("edit_distance", edit_distance)

<class 'networkx.classes.digraph.DiGraph'>
<class 'networkx.classes.digraph.DiGraph'>
396.0
395.0
393.0


## PageRank

In [4]:
# rater1_graph = create_graphs(raters['Frosi'])
# print(type(rater1_graph))
# rater2_graph = create_graphs(raters['Chiara'])
# print(type(rater2_graph))
# edit_path_distance=0
# degree=nx.degree_histogram(rater1_graph)
# sp = dict(nx.all_pairs_shortest_path(rater1_graph))
# pagerank=nx.pagerank(rater1_graph)
results_list=[]

for rater in raters:
    rater_graph = create_graphs(raters[rater])
    pagerank=nx.pagerank(rater_graph)
    for k,v in pagerank.items():

        if rater=="Samu":
            new_raw = {"concept":k,"Samu":v, "Chiara":'', "Frosi":'',"Andrea":'',"Marco":''}
            results_list.append(new_raw)
        elif rater=="Chiara":
            new_raw = {"concept":k,"Samu":'', "Chiara":v, "Frosi":'',"Andrea":'',"Marco":''}
            results_list.append(new_raw)
        elif rater=="Frosi":
            new_raw = {"concept":k,"Samu":'', "Chiara":'', "Frosi":v,"Andrea":'',"Marco":''}
            results_list.append(new_raw)
        elif rater=="Andrea_Unique":
            new_raw = {"concept":k,"Samu":'', "Chiara":'', "Frosi":'',"Andrea":v,"Marco":''}
            results_list.append(new_raw)
        elif rater=="Marco":
            new_raw = {"concept":k,"Samu":'', "Chiara":'', "Frosi":'',"Andrea":'',"Marco":v}
            results_list.append(new_raw)


pagerank_df = pd.DataFrame(results_list,columns=["concept","Samu", "Chiara", "Frosi","Andrea","Marco"])
pagerank_df.to_csv("pagerank.csv", sep=";", encoding="utf-8")

# optimize_edit_paths=nx.optimize_edit_paths(rater1_graph, rater2_graph, node_match=lambda node1, node2:node1['label']==node2['label'])
#
# print("optimize_edit_paths")
# for i in optimize_edit_paths:
#     print(i,"\n")

print("pagerank: ", pagerank)

pagerank:  {'NETWORK SOFTWARE': 0.00949903446862644, 'PROPRIETARY SYSTEM': 0.015249089204274438, 'OWNERSHIP RIGHT': 0.020137212207672193, 'BUS NETWORK': 0.010480748059705784, 'SHORT BUS': 0.009402319758519961, 'INTERNET PROTOCOL ADDRESS': 0.011952940662457215, 'DOTTED DECIMAL NOTATION': 0.01733925320080033, 'INFORMATION': 0.007175150244163736, 'COMPUTER': 0.0132738804399433, 'INTERNET': 0.014355778597997061, 'ADDRESSING SYSTEM': 0.011242601097595468, 'DOMAIN NAME': 0.011952940662457215, 'SOFTWARE': 0.010936110013015168, 'TOPOLOGY': 0.007175150244163736, 'ETHERNET NETWORK': 0.010480748059705784, 'STAR NETWORK': 0.010480748059705784, 'DISTRIBUTED COMPUTE': 0.007175150244163736, 'CLUSTER COMPUTE': 0.009208060309423591, 'GRID COMPUTE': 0.009208060309423591, 'CLOUD COMPUTING': 0.009208060309423591, 'NETWORK PROTOCOL': 0.007175150244163736, 'NNTP': 0.008191605276793664, 'TELNET': 0.008191605276793664, 'SIMPLE MAIL TRANSFER PROTOCOL': 0.008191605276793664, 'EMAIL MESSAGE': 0.01413790529173908

## Hubs and Authorities

In [13]:
rater1_graph = create_graphs(raters['Chiara'])
h,a=nx.hits(rater1_graph)
print("hits",h)
print("authorities",a)
with open('hits_chiara.csv', 'w') as f:
    for key in h.keys():
        f.write("%s,%s\n"%(key,h[key]))

with open('authorities_chiara.csv', 'w') as f:
    for key in a.keys():
        f.write("%s,%s\n"%(key,a[key]))

hits {'ACCESS INTERNET SERVICE PROVIDER': 0.006048943749040595, 'HOST': 0.021438583681552256, 'GATEWAY': 0.0051290503221034945, 'VOICE OVER INTERNET PROTOCOL': 0.0, 'BROWSER': 0.012701150566432496, 'WEBSERVER': 0.006231811709115577, 'COMPUTER': 0.14695835643421706, 'HYPERTEXT TRANSFER PROTOCOL': 0.005515679435961098, 'COMMON GATEWAY INTERFACE': 0.0012721802376263408, 'SERVLETS': 0.0, 'SERVER': 0.03392628347256312, 'SERVER SIDE': 0.0013185221265762893, 'SERVLET': 0.0, 'ACTIVE SERVER PAGE': 2.337452514172505e-05, 'ASP': 0.0, 'BUS': 0.00028500125833242517, 'STAR': 1.4159668848142142e-40, 'MACHINE': 0.05347196746579334, 'PROTOCOL': 0.028317107914718625, 'ETHERNET NETWORK': 0.000781949444563868, 'CARRIER SENSE': 0.004835050025698207, 'INTERNET ACCESS': 0.0, 'ACCESS POINT': 0.011314612088107731, 'END SYSTEM': 0.00045814950028630634, 'INTERNET SERVICE PROVIDER': 0.004455420099715446, 'WIFI NETWORK': 0.0040245546883293, 'INTERNET MAIL ACCESS PROTOCOL': 0.0, 'TRANSPORT LAYER': 0.004843418817172